In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

Let us start by importing the necessary libraries.

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('fivethirtyeight')
from scipy import stats
import sklearn
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import warnings
warnings.filterwarnings('ignore')


from scipy import stats
from scipy.stats import norm, skew

In [ ]:
train_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/train.csv')
test_data = pd.read_csv('../input/house-prices-advanced-regression-techniques/test.csv')

In [ ]:
train_data.shape


In [ ]:
test_data.shape

In [ ]:
train_data.head()

In [ ]:
test_data.head()

In [ ]:
train_data.columns

Let us look at the columns a bit closely.

* SalePrice - the property's sale price in dollars. This is the target variable that you're trying to predict.
* MSSubClass: The building class
* MSZoning: The general zoning classification
* LotFrontage: Linear feet of street connected to property
* LotArea: Lot size in square feet
* Street: Type of road access
* Alley: Type of alley access
* LotShape: General shape of property
* LandContour: Flatness of the property
* Utilities: Type of utilities available
* LotConfig: Lot configuration
* LandSlope: Slope of property
* Neighborhood: Physical locations within Ames city limits
* Condition1: Proximity to main road or railroad
* Condition2: Proximity to main road or railroad (if a second is present)
* BldgType: Type of dwelling
* HouseStyle: Style of dwelling
* OverallQual: Overall material and finish quality
* OverallCond: Overall condition rating
* YearBuilt: Original construction date
* YearRemodAdd: Remodel date
* RoofStyle: Type of roof
* RoofMatl: Roof material
* Exterior1st: Exterior covering on house
* Exterior2nd: Exterior covering on house (if more than one material)
* MasVnrType: Masonry veneer type
* MasVnrArea: Masonry veneer area in square feet
* ExterQual: Exterior material quality
* ExterCond: Present condition of the material on the exterior
* Foundation: Type of foundation
* BsmtQual: Height of the basement
* BsmtCond: General condition of the basement
* BsmtExposure: Walkout or garden level basement walls
* BsmtFinType1: Quality of basement finished area
* BsmtFinSF1: Type 1 finished square feet
* BsmtFinType2: Quality of second finished area (if present)
* BsmtFinSF2: Type 2 finished square feet
* BsmtUnfSF: Unfinished square feet of basement area
* TotalBsmtSF: Total square feet of basement area
* Heating: Type of heating
* HeatingQC: Heating quality and condition
* CentralAir: Central air conditioning
* Electrical: Electrical system
* 1stFlrSF: First Floor square feet
* 2ndFlrSF: Second floor square feet
* LowQualFinSF: Low quality finished square feet (all floors)
* GrLivArea: Above grade (ground) living area square feet
* BsmtFullBath: Basement full bathrooms
* BsmtHalfBath: Basement half bathrooms
* FullBath: Full bathrooms above grade
* HalfBath: Half baths above grade
* Bedroom: Number of bedrooms above basement level
* Kitchen: Number of kitchens
* KitchenQual: Kitchen quality
* TotRmsAbvGrd: Total rooms above grade (does not include bathrooms)
* Functional: Home functionality rating
* Fireplaces: Number of fireplaces
* FireplaceQu: Fireplace quality
* GarageType: Garage location
* GarageYrBlt: Year garage was built
* GarageFinish: Interior finish of the garage
* GarageCars: Size of garage in car capacity
* GarageArea: Size of garage in square feet
* GarageQual: Garage quality
* GarageCond: Garage condition
* PavedDrive: Paved driveway
* WoodDeckSF: Wood deck area in square feet
* OpenPorchSF: Open porch area in square feet
* EnclosedPorch: Enclosed porch area in square feet
* 3SsnPorch: Three season porch area in square feet
* ScreenPorch: Screen porch area in square feet
* PoolArea: Pool area in square feet
* PoolQC: Pool quality
* Fence: Fence quality
* MiscFeature: Miscellaneous feature not covered in other categories
* MiscVal: Value of miscellaneous feature
* MoSold: Month Sold
* YrSold: Year Sold
* SaleType: Type of sale
* SaleCondition: Condition of sale

# **EDA**

## Handling Null Values

We find out the columns containing null values.

> **Why?** : These null values adversely affect the performance and accuracy of any machine learning algorithm. 



In [ ]:
# Finding the null values columnwise
na_vals=train_data.isna().sum()
na_cols = na_vals.drop(na_vals[na_vals==0].index).sort_values(ascending=False)

na_test_vals=test_data.isna().sum()
na_test_cols = na_test_vals.drop(na_test_vals[na_test_vals==0].index).sort_values(ascending=False)

na_cols

In [ ]:
na_cols.dtype

In [ ]:
percent = (train_data.isnull().sum()/train_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([na_vals, percent], axis=1, keys=['Total', 'Percent'])
missing_data = missing_data.drop(na_vals[na_vals==0].index)
missing_data=missing_data.sort_values('Percent',ascending=False)
missing_data

In [ ]:
percent = (test_data.isnull().sum()/test_data.isnull().count()).sort_values(ascending=False)
missing_test_data = pd.concat([na_test_vals, percent], axis=1, keys=['Total', 'Percent'])
missing_test_data = missing_test_data.drop(na_test_vals[na_test_vals==0].index)
missing_test_data=missing_test_data.sort_values('Percent',ascending=False)
print(missing_test_data)

test_data['BsmtFullBath'].fillna(test_data['BsmtFullBath'].mode()[0], inplace=True)
test_data['BsmtHalfBath'].fillna(test_data['BsmtHalfBath'].mode()[0], inplace=True)
test_data['BsmtFinSF1'].fillna(test_data['BsmtFinSF1'].mode()[0], inplace=True)
test_data['BsmtFinSF2'].fillna(test_data['BsmtFinSF2'].mode()[0], inplace=True)
test_data['TotalBsmtSF'].fillna(test_data['TotalBsmtSF'].mode()[0], inplace=True)
test_data['GarageArea'].fillna(test_data['GarageArea'].mode()[0], inplace=True)
test_data['BsmtUnfSF'].fillna(test_data['BsmtUnfSF'].mode()[0], inplace=True)


Based on the above percentage, we drop the columns if the percentage of missing values is more than 0.1.

In [ ]:
train_data = train_data.drop((missing_data[missing_data['Percent'] > 0.1]).index,1)


In [ ]:
test_data = test_data.drop((missing_test_data[missing_test_data['Percent'] > 0.1]).index,1)

Since we do not need the column 'ID', we will drop the column after storing it for using later.

In [ ]:
train_ID = train_data['Id']
test_ID = test_data['Id']

train_data.drop("Id", axis = 1, inplace = True)
test_data.drop("Id", axis = 1, inplace = True)

In [ ]:
train_data.shape

In [ ]:
test_data.shape

Since 'GarageType', 'GarageFinish', 'GarageQual' and 'GarageCond' are categorical values, we will fill their missing values with 'None', and those of 'GarageYrBlt', 'GarageCars' are numeric, so them with '0'. 

In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    train_data[col] = train_data[col].fillna('None')


for col in ('GarageYrBlt', 'GarageCars'):
    train_data[col] = train_data[col].fillna(0)   



In [ ]:
for col in ('GarageType', 'GarageFinish', 'GarageQual', 'GarageCond'):
    test_data[col] = test_data[col].fillna('None')

for col in ('GarageYrBlt', 'GarageCars'): 
    test_data[col] = test_data[col].fillna(0)


Similarly for the columns 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2').

In [ ]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    train_data[col] = train_data[col].fillna('None')

In [ ]:
for col in ('BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2'):
    test_data[col] = train_data[col].fillna('None')

In [ ]:
train_data["MasVnrType"] = train_data["MasVnrType"].fillna("None")
train_data["MasVnrArea"] = train_data["MasVnrArea"].fillna(0)

In [ ]:
test_data["MasVnrType"] = test_data["MasVnrType"].fillna("None")
test_data["MasVnrArea"] = test_data["MasVnrArea"].fillna(0)

In [ ]:
train_data['Electrical'] = train_data['Electrical'].fillna(train_data['Electrical'].mode()[0])

In [ ]:
test_data['Electrical'] = test_data['Electrical'].fillna(test_data['Electrical'].mode()[0])

## Outlier Detection 
#### Scatter Plot and Box Plot

> **Why ?** Outliers are unusual values in the dataset, and they can distort statistical analyses and violate their assumptions. 

In [ ]:
sns.boxplot(y='SalePrice', x = 'OverallQual', data=train_data)

Let us look for outliers in columns like GrLivArea. We do this using **scatterplot**.

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
ax.scatter(train_data['GrLivArea'], train_data['SalePrice'])
ax.set_ylabel('SalePrice')
ax.set_xlabel('GrLivArea')
plt.show()

The points having GrLivArea greater that 4000 are outliers for this case, and hence we drop those values.

In [ ]:
train_data = train_data.drop(train_data[(train_data['GrLivArea']>4000)].index)

If we check again, these outliers have now been removed.

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
ax.scatter(train_data['GrLivArea'], train_data['SalePrice'])
ax.set_ylabel('SalePrice')
ax.set_xlabel('GrLivArea')
plt.show()

In [ ]:
fig, ax = plt.subplots(figsize=(7,5))
ax.scatter(train_data['GarageArea'], train_data['SalePrice'])
ax.set_ylabel('SalePrice')
ax.set_xlabel('GarageArea')
plt.show()

Since there are no extreme outliers, it is preferred not to drop any values. 

Let us also look at the scatter plots of other columns against 'SalePrice'.

In [ ]:
training_data=train_data.copy()
training_data.drop(['SalePrice'],axis=1)
data = pd.concat([training_data,test_data], axis=0)
data.drop(['SalePrice'],axis=1,inplace=True)

In [ ]:
numRows = 7
numCols = 5
 
fig, axes = plt.subplots(numRows, numCols,figsize=(20,21))
palette= sns.color_palette("Paired", 40)

numerical_cols = training_data.dtypes[training_data.dtypes != "object"].index
listNum = list(numerical_cols)

x=-1
for num in range(0, numRows):
    for col in range(0, numCols):  
        i = num * numCols + col
        if i < len(listNum):
            x += 1
            sns.regplot(training_data[listNum[i]], training_data['SalePrice'], ax = axes[num][col],color = palette[x],marker=".")  
plt.tight_layout()    
plt.show()   

## Label Encoding 

Before we apply label encoding, let us look at the categorical variables at hand and their composition.

In [ ]:
from sklearn.preprocessing import LabelEncoder
data = training_data.copy()
categorical_features= data.select_dtypes(include=['object']).copy()
number=[len(data[features].unique()) for features in categorical_features]
data_tuples = list(zip(categorical_features,number))
categorical_data= pd.DataFrame(data_tuples, columns=['Features','Number of distinct values '])
categorical_data


In [ ]:
#Visualizing the values in 'Neighborhood'
plt.figure(figsize=(12,8))
n_count = data['Neighborhood'].value_counts()
plot=sns.barplot(n_count.index, n_count.values)
plt.setp(plot.get_xticklabels(), rotation=30, fontsize=10)
plt.title('Distribution of Neighborhood values ')
plt.ylabel('Number of Occurrences', fontsize=12)
plt.xlabel('Values', fontsize=12)
plt.show()

In [ ]:
pieplot = data.groupby('HouseStyle').size()

pieplot.plot(kind='pie', subplots=True, figsize=(7, 7), fontsize=9)
plt.title("Pie Chart of HouseStyle")
plt.ylabel("")

In [ ]:
for c in categorical_features:
    label_encoder = LabelEncoder() 
    label_encoder.fit(list(data[c].values)) 
    data[c] = label_encoder.transform(list(data[c].values))
    
training_data=data.copy()

In [ ]:
data = test_data.copy()
categorical_features = [features for features in data.columns if data[features].dtype == 'O']
    
for c in categorical_features:
    label_encoder = LabelEncoder() 
    label_encoder.fit(list(data[c].values)) 
    data[c] = label_encoder.transform(list(data[c].values))
    
test_data=data

# Checking the Four Assumptions

Before moving on to building the regression model, we must test the assumptions underlying the statistical bases for multivariate analysis.

#### **1. Correlation**

Multicollinearity occurs when independent variables in a regression model are correlated. This correlation is a problem because independent variables should be independent. If the degree of correlation between variables is high enough, it can cause problems when you fit the model and interpret the results.

> **Why?** When independent variables are correlated, it indicates that changes in one variable are associated with shifts in another variable. The stronger the correlation, the more difficult it is to change one variable without changing another.

In [ ]:
corrmat = train_data.corr()
plt.subplots(figsize=(25,25))
sns.heatmap(corrmat,fmt = ".1f", cmap = "coolwarm")

#### More correlation

We check for high correlation in the following steps.

In [ ]:
def getCorrelatedFeature(corrdata, threshold):
    feature = []
    value = []
    for i , index in enumerate(corrdata.index):
        if abs(corrdata[index]) > threshold:
            feature.append(index)
            value.append(corrdata[index])
    df2 = pd.DataFrame(data = value, index=feature, columns=['corr value'] )
    return df2

corr_df = getCorrelatedFeature(corrmat['SalePrice'], 0.5)
corr_df.sort_values(by=['corr value'],ascending=False)

#### **2. Linearity**

The most common way to assess linearity is to examine scatter plots and search for linear patterns. If patterns are not linear, it would be worthwhile to explore data transformations. However, we'll not get into this because most of the scatter plots we've seen above appear to have linear relationships.

>**Why?** If you fit a linear model to a non-linear, non-additive data set, the regression algorithm would fail to capture the trend mathematically, thus resulting in an inefficient model.

#### **3. Homoscedasticity**

Homoscedasticity refers to the assumption that dependent variable(s) exhibit equal, constant levels of variance across the range of predictor variable(s).

>**Why?** If the variance of the errors is increasing over time, confidence intervals for out-of-sample predictions will tend to be unrealistically narrow. It  may also have the effect of giving too much weight to a small subset of the data when estimating coefficients.

In [ ]:
plt.scatter(train_data.GrLivArea, train_data.SalePrice);

We observe that there is heteroscedasticity present. 

(The plot should look like a tube/pipe, not a funnel.)

We try to remove it in the following code.


#### **4. Normality of Errors**

The data should look like a normal distribution. Several statistic tests, like the t-tests, rely on the normal distribution of error terms.

>**Why?** Calculation of confidence intervals and various significance tests for coefficients are all based on the assumptions of normally distributed errors. If the error distribution is significantly non-normal, confidence intervals may be too wide or too narrow.

In [ ]:
print("Skewness: %f" % train_data['SalePrice'].skew())
print("Kurtosis: %f" % train_data['SalePrice'].kurt())
print()
fig, ax = plt.subplots(1,2, figsize=(16,4))
sns.distplot(train_data['SalePrice'] , fit=norm, ax=ax[0])
#Get also the QQ-plot
res = stats.probplot(train_data['SalePrice'],plot=ax[1])
plt.show()

The figures show that the target variable has peakedness, and the QQ plot shows that the target variable "SalePrice" has right (positive) skewness.

We will apply log transformation to make the variables normally distributed, i.e, achieve normality of errors.

In [ ]:
train_data['SalePrice'] = np.log(train_data['SalePrice'])

In [ ]:
print("Skewness: %f" % train_data['SalePrice'].skew())
print("Kurtosis: %f" % train_data['SalePrice'].kurt())
print()
fig, ax = plt.subplots(1,2, figsize=(12,4))
sns.distplot(train_data['SalePrice'] , fit=norm, ax=ax[0])
#Get also the QQ-plot
res = stats.probplot(train_data['SalePrice'],plot=ax[1])
plt.show()

Now let's come back to check the difference upon homoscedasticity. 

In [ ]:
plt.scatter(train_data.GrLivArea, train_data.SalePrice);

The heteroscedasticity has been removed now, owing to the log transformation that we put 'Saleprice' through.

Note : Heteroscedasticity where the spread is close to proportional to the conditional mean will tend to be improved by taking log(y), but if it's not increasing with the mean at close to that rate (or more), then the heteroscedasticity will often be made worse by that transformation.

### Box-Cox Transformation 

We will now apply box-cox transformation on the training data so as to normalize our training attributes as well.

In [ ]:
#Box-cox transformation on training_data and test_data
numeric_feats =training_data.dtypes[training_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = training_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
print("\nSkew in numerical features: \n")
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness.head(10)


In [ ]:
skewness = skewness[abs(skewness) > 0.75]

from scipy.special import boxcox1p
skewed_features = skewness.index
lambda_value = 0.15
for feat in skewed_features:
    training_data[feat] = boxcox1p(training_data[feat], lambda_value)


In [ ]:
numeric_feats =test_data.dtypes[test_data.dtypes != "object"].index

# Check the skew of all numerical features
skewed_feats = test_data[numeric_feats].apply(lambda x: skew(x.dropna())).sort_values(ascending=False)
skewness = pd.DataFrame({'Skew' :skewed_feats})
skewness = skewness[abs(skewness) > 0.75]

skewed_features = skewness.index
lambda_value = 0.15
for feat in skewed_features:
    test_data[feat] = boxcox1p(test_data[feat], lambda_value)


In [ ]:
# final_data=train_data.copy()
# final_data.drop(['SalePrice'],axis=1)
# X_train=final_data
# X_train= X_train.drop(['SalePrice'],axis=1)
# y_train=train_data['SalePrice'].values

# feature_sel_model = SelectFromModel(Lasso(alpha=0.015,random_state=0))
# feature_sel_model.fit(X_train,y_train)
# feature_sel_model.get_support()

# total_selected=X_train.columns[(feature_sel_model.get_support())]
# len(total_selected)

In [ ]:
final_data=training_data.copy()
final_data.drop(['SalePrice'],axis=1)
X_train=final_data
X_train= X_train.drop(['SalePrice'],axis=1)
y_train=train_data['SalePrice'].values
X_test=test_data

# Building the Model

In [ ]:
from sklearn.model_selection import KFold,cross_val_score
from sklearn.metrics import make_scorer,r2_score
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import Lasso


def test_model(model, X_train=X_train, y_train=y_train):
    cv = KFold(n_splits=7,shuffle=True,random_state=42)
    r2 = make_scorer(r2_score)
    r2_val_score = cross_val_score(model,X_train,y_train,cv=cv,scoring=r2)
    score = [r2_val_score.mean()]
    return score

#### Linear Regression 

In [ ]:
from sklearn.linear_model import LinearRegression, Ridge
LR = LinearRegression(normalize=True, n_jobs=-1)
LR.fit(X_train,y_train)
test_model(LR)

#### Ridge Regression 

In [ ]:
RidR = Ridge(alpha = 0.5, normalize=True)
RidR.fit(X_train,y_train)
test_model(RidR)

#### Lasso Regression

In [ ]:
LasR = Lasso(alpha = 0.001, normalize=True)
LasR.fit(X_train,y_train)
test_model(LasR)

#### Decision Tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor

DTree = DecisionTreeRegressor(max_depth=8)
DTree.fit(X_train,y_train)
test_model(DTree)

#### Random Forest 

In [ ]:
from sklearn.ensemble import RandomForestRegressor
RF = RandomForestRegressor(random_state = 42)
RF.fit(X_train,y_train)
test_model(RF)

#### ElasticNet

In [ ]:
from sklearn.linear_model import ElasticNet
ENet = ElasticNet(alpha=0.0005, l1_ratio=.9, random_state=3)
ENet.fit(X_train,y_train)
test_model(ENet)

#### XGBoost

In [ ]:
import xgboost
xgb = xgboost.XGBRegressor(learning_rate=0.05, max_depth=3, random_state=4)
xgb.fit(X_train,y_train)
test_model(xgb)

#### Support Vector Machine

In [ ]:
from sklearn.svm import SVR
Svm = SVR(kernel='rbf')
Svm.fit(X_train,y_train)
test_model(Svm)

#### Light Gradient Boosting

In [ ]:
import lightgbm as lgb
model_lgb = lgb.Dataset(X_train, y_train)
params ={ 'objective':'regression',
        'task': 'train',
        'boosting_type': 'gbdt',
        'num_leaves': 42,
        'metric': {'15'},
        'learning_rate': 0.1, 
        'bagging_fraction': 0.8,
        'bagging_freq' : 5,
        'feature_fraction':0.9, 
        'min_data_in_leaf' : 6,
        } 
prediction = lgb.train(params, model_lgb)
y_predict = prediction.predict(X_test)
y_predict = np.expm1(y_predict)

In [ ]:
pred=np.exp(prediction.predict(X_test)).round(2)

In [ ]:
submit_test = pd.concat([test_ID,pd.DataFrame(pred)],axis=1)
submit_test.columns = ['Id','SalePrice']

In [ ]:
submit_test.to_csv('submission.csv', index=False)